In [30]:
import pickle
from pathlib import Path
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import os

data_path = Path("/home/group01/mcv/datasets/MIT_split")
feature_path = Path("./results/retrieval")

In [7]:
with open(feature_path / "queries.npy", "rb") as f:
    query_data = np.load(f)
with open(feature_path / "catalogue.npy", "rb") as f:
    catalogue_data = np.load(f)

with (feature_path / "catalogue_meta.pkl").open('rb') as fc, \
        (feature_path / "query_meta.pkl").open('rb') as fq:
    catalogue_meta = pickle.load(fc)
    query_meta = pickle.load(fq)

In [29]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


def pk(actual, predicted, k=10):
    """
    Computes the precision at k.
    This function computes the precision at k between the query image and a list
    of database retrieved images.
    Parameters
    ----------
    actual : int
             The element that has to be predicted
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The precision at k over the input
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0
    for i in range(len(predicted)):
        if actual == predicted[i]:
            score += 1
    
    return score / len(predicted)

def mpk(actual, predicted, k=10):
    """
    Computes the precision at k.
    This function computes the mean precision at k between a list of query images and a list
    of database retrieved images.
    Parameters
    ----------
    actual : list
             The query elements that have to be predicted
    predicted : list
                A list of predicted elements (order does matter) for each query element
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The precision at k over the input
    """
    pk_list = []
    for i in range(len(actual)):
        score = pk(actual[i], predicted[i], k)
        pk_list.append(score)
    return np.mean(pk_list)

def AP(actual, predicted):
    """
    Computes the precision at k.
    This function computes the average precision precision between the query image and a list
    of database retrieved images.
    Parameters
    ----------
    actual : int
             The element that has to be predicted
    predicted : list
                A list of predicted elements (order does matter)
    Returns
    -------
    score : double
            The Average Precision over the input
    """
    gtp = 0
    ap = 0
    for i in range(len(predicted)):
        a = pk(actual, predicted, i+1)
        if actual == predicted[i]: 
            b = 1
            gtp += 1
        else: 
            b = 0
        c = a*b
        ap += c

    return ap/gtp

def mAP(actual, predicted):
    """
    Computes the precision at k.
    This function computes the mean Average Precision between the query image and a list
    of database retrieved images.
    Parameters
    ----------
    actual : list
             The query elements that have to be predicted
    predicted : list
                A list of predicted elements (order does matter) for each query element
    Returns
    -------
    score : double
            The mean Average Precision over the input
    """
    ap_list = []
    for i in range(len(actual)):
        ap = AP(actual[i], predicted[i])
        ap_list.append(ap)
    return np.mean(ap_list)

### K-nn

In [32]:
with open(feature_path / "Knn_results.pkl", "rb") as f:
        neighbors = pickle.load(f)

In [33]:
neighbors_labels = []
for i in range(len(neighbors)):
    neighbors_class = [catalogue_meta[j][1] for j in neighbors[i]]
    neighbors_labels.append(neighbors_class)

query_labels = [x[1] for x in query_meta]

p_1 = mpk(query_labels,neighbors_labels, 1)
p_5 = mpk(query_labels,neighbors_labels, 5)
print('P@1=',p_1)
print('P@5=',p_5)

map = mAP(query_labels,neighbors_labels)
print('mAP=',map)

P@1= 0.3395291201982652
P@5= 0.3132589838909542
mAP= 0.21145447868800407


Calculate metrics for each class individually

In [38]:
labels = ['Open Country', 'Coast', 'Forest', 'Highway', 'Inside City', 'Mountain', 'Street', 'Tall Building']

class_metrics = {}
for i in range(len(labels)):
    predictions = []
    gt = []
    for j in range(len(query_labels)):
        if query_labels[j]==i:
            predictions.append(neighbors_labels[j])
            gt.append(query_labels[j])
    
    p_1 = mpk(gt,predictions, 1)
    p_5 = mpk(gt,predictions, 5)
    print('Class: ', labels[i])
    print('P@1=',p_1)
    print('P@5=',p_5)
    map = mAP(gt,predictions)
    print('mAP=',map)
    class_metrics[labels[i]] = {'p_1': p_1, 'p_5': p_5, 'mAP': map}

Class:  Open Country
P@1= 0.2033898305084746
P@5= 0.22033898305084743
mAP= 0.2456416325528106
Class:  Coast
P@1= 0.07758620689655173
P@5= 0.08793103448275862
mAP= 0.12728370312180862
Class:  Forest
P@1= 0.5148514851485149
P@5= 0.4138613861386139
mAP= 0.24018926144929248
Class:  Highway
P@1= 0.2236842105263158
P@5= 0.22368421052631585
mAP= 0.1195978984157583
Class:  Inside City
P@1= 0.4148936170212766
P@5= 0.35319148936170214
mAP= 0.24333730889777633
Class:  Mountain
P@1= 0.4473684210526316
P@5= 0.4315789473684211
mAP= 0.2517056224798895
Class:  Street
P@1= 0.4125
P@5= 0.3325
mAP= 0.18972434741864375
Class:  Tall Building
P@1= 0.4537037037037037
P@5= 0.4518518518518519
mAP= 0.2481341724008498


In [39]:
print(class_metrics)

{'Open Country': {'p_1': 0.2033898305084746, 'p_5': 0.22033898305084743, 'mAP': 0.2456416325528106}, 'Coast': {'p_1': 0.07758620689655173, 'p_5': 0.08793103448275862, 'mAP': 0.12728370312180862}, 'Forest': {'p_1': 0.5148514851485149, 'p_5': 0.4138613861386139, 'mAP': 0.24018926144929248}, 'Highway': {'p_1': 0.2236842105263158, 'p_5': 0.22368421052631585, 'mAP': 0.1195978984157583}, 'Inside City': {'p_1': 0.4148936170212766, 'p_5': 0.35319148936170214, 'mAP': 0.24333730889777633}, 'Mountain': {'p_1': 0.4473684210526316, 'p_5': 0.4315789473684211, 'mAP': 0.2517056224798895}, 'Street': {'p_1': 0.4125, 'p_5': 0.3325, 'mAP': 0.18972434741864375}, 'Tall Building': {'p_1': 0.4537037037037037, 'p_5': 0.4518518518518519, 'mAP': 0.2481341724008498}}
